In [10]:
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import json
from newsapi import NewsApiClient
from nltk.corpus import reuters
import os
import pandas as pd
from pathlib import Path
import re
import requests

load_dotenv()

True

## 2021 Global 100 - Most Sustainable Companies in the World



In [11]:
# Read Global 100 companies CSV file
global100_filepath = Path("data/2021-Global-100.csv")
global100_df = pd.read_csv(global100_filepath)
global100_df.head()

,Rank 2021,Rank 2020,Name,Peer Group (CKIG),Peer Group Rank,Country,Energy Productivity,Energy Productivity Score,Carbon Productivity,Carbon Productivity Score,...,CEO-Average Worker Pay ratio - Weight,Clean Revenue - Weight,Clean Investment - Weight,Non-male on Board - Weight,Non-male in Executive Management - Weight,Racial Diversity Among Executives - Weight,Racial Diversity on Board of Directors - Weight,Paid Sick Leave,Sustainability Paylink - Weight,Supplier Score - Weight
0,1,29.0,Schneider Electric SE,Industrial Conglomerates,#1/53,France,"$ 13,989",96.0%,"$ 90,981",66.9%,...,3%,43%,8%,3%,3%,3%,3%,3%,5%,3%
1,2,1.0,Orsted A/S,Power Generation,#1/244,Denmark,$ 467,39.7%,"$ 5,361",63.7%,...,0%,43%,8%,3%,3%,3%,3%,3%,5%,3%
2,3,9.0,Banco do Brasil SA,Banks and Investment Services,#1/935,Brazil,"$ 232,205",72.0%,"$ 851,966",32.5%,...,4%,43%,8%,3%,3%,3%,3%,3%,5%,3%
3,4,3.0,Neste Oyj,Oil & Gas,#1/347,Finland,$ 553,76.6%,"$ 7,765",70.1%,...,0%,43%,8%,3%,3%,3%,3%,3%,5%,3%
4,5,57.0,Stantec Inc,Consulting and Professional Services,#1/23,Canada,"$ 5,418",20.8%,"$ 64,411",18.2%,...,10%,43%,8%,3%,3%,3%,3%,3%,5%,3%


In [12]:
# Drop empty column
global100_df.drop(columns=["Unnamed: 56"], inplace=True)

# Trim whitespaces from column names
global100_df.columns = [re.sub("\s+", " ", col.strip()) for col in global100_df.columns]

In [13]:
# Convert some columns to numeric
numeric_column_names = ["Energy Productivity", "Carbon Productivity", "Water Productivity", "Waste Productivity", "VOC Productivity", "NOx Productivity", "SOx Productivity", "PM Productivity", "Retirement contributions/FTE", "Fair value of plan assets/FTE", "Energy Productivity Score", "Carbon Productivity Score", "Water Productivity Score", "Waste Productivity Score", "VOC Productivity Score", "NOx Productivity Score", "SOx Productivity Score", "PM Productivity Score", "% Taxes Paid", "% Taxes Paid Score", "% Clean Revenues", "Clean Revenue Score", "% Clean Investment", "Clean Investment Score", "CEO-Average Worker Pay Score", "Retirement contributions/FTE Score", "Fair value of plan assets/FTE Score", "Fair value/projected benefit obligation Percent", "Fair value/projected benefit obligation Score", "Pension Fund Status", "Lost-time Injury Rate Score", "Fatalities per FTE employee Score", "Employee Turnover Rate", "Employee Turnover Score", "% Non-male on Board", "% Non-male on Board Score", "% Non-male Senior Executives", "% Non-male Senior Executives Score", "% Racial Diversity Among Boards", "% Racial Diversity Among Board Score", "% Racial Diversity Among Executives", "% Racial Diversity Among Executives Score", "Paid Sick Leave Score", "Sustainability Paylink Score", "Supplier Score", "Fines deduction", "Overall Score", "Energy Productivity - Weight", "Carbon Productivity - Weight", "Water Productivity - Weight", "Waste Productivity - Weight", "Injuries - Weight", "Fatalities - Weight", "Employee turnover - Weight", "VOC Productivity - Weight", "NOX Productivity - Weight", "SOX Productivity - Weight", "PM Productivity - Weight", "Pension Fund Status - Weight", "Cash taxes paid ratio - Weight", "CEO-Average Worker Pay ratio - Weight", "Clean Revenue - Weight", "Clean Investment - Weight", "Non-male on Board - Weight", "Non-male in Executive Management - Weight", "Racial Diversity Among Executives - Weight", "Racial Diversity on Board of Directors - Weight", "Paid Sick Leave", "Sustainability Paylink - Weight", "Supplier Score - Weight", "Lost-time Injury Rate", "Fatalities per FTE employee"]

for column_name in numeric_column_names:
    global100_df[column_name] = global100_df[column_name].replace(to_replace="[^\d.]+", value="", regex=True)
    global100_df[column_name] = pd.to_numeric(global100_df[column_name], errors="coerce")

In [14]:
# Convert ratio column
global100_df["CEO-Average Worker Pay Ratio"] = pd.to_numeric(global100_df["CEO-Average Worker Pay Ratio"].apply(lambda v: v.split(":")[0]), errors="coerce")

In [15]:
# Split peer group rank column into 2
global100_df["Peer Group Rank Total"] = pd.to_numeric(global100_df["Peer Group Rank"].apply(lambda v: v.split("/")[1]), errors="coerce")
global100_df["Peer Group Rank"] = pd.to_numeric(global100_df["Peer Group Rank"].apply(lambda v: v.split("/")[0].replace("#", "")), errors="coerce")

## 2020 Fortune's Change the World List

Top businesses changing the world

In [16]:
# read JSON file
change_the_world_2020_filepath = Path("data/change_the_world_2020.json")
with open(change_the_world_2020_filepath, "r") as datafile:
    data = datafile.read()

json_data = json.loads(data)

rows = []
for item in json_data:
    obj = {}
    for field in item["fields"]:
        obj[field["key"]] = field["value"]
    rows.append(obj)

change_the_world_2020_df = pd.DataFrame(rows)

# Export to CSV
change_the_world_2020_output_filepath = Path("data/change_the_world_2020.csv")
change_the_world_2020_df.to_csv(change_the_world_2020_output_filepath, index=False)